In [1]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import re
import pandas as pd
import os
import numpy as np
from janitor import clean_names
from google.cloud import bigquery
import pandas_gbq

In [2]:
archive_link = 'https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page'
h = httplib2.Http()
status, response = h.request(archive_link)

a_tags = BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a'))
links = [link['href'] for link in a_tags if link.has_attr('href')]

In [3]:
is_excel = lambda x: (x.endswith('xlsx') or x.endswith('xls')) and 'neighborhood' not in x

f_names=list(filter(is_excel, links))

paths=["https://www1.nyc.gov/" +  f for f in f_names]

In [4]:
yr = re.compile('\d{2,4}(?!pdf)')

get_year = lambda x: yr.findall(x)[0]

f2=list(map(get_year, f_names))

yrs=[int('20' + x) if len(x) == 2 else int(x) for x in f2]
u_yrs=np.unique(yrs)

In [5]:
f_dict={}
for y in range(min(yrs), max(yrs)+1):
    f_list = []
    for i in range(0, len(paths)):
        if yrs[i] == y:
            f_list.append(paths[i])
    f_dict[str(y)] = f_list 
    

In [6]:
def clean_excel(file:str)->pd.DataFrame:
    d=pd.read_excel(file)
    d2=d[~d.iloc[:,1].isnull()]
    d2.iloc[0, :].str.replace('\n', '')
    cols=list(d2.iloc[0, :].str.replace('\n', ' '))
    d2.columns = cols
    d3=d2.iloc[1:,:].clean_names()
    return d3 

In [7]:
def clean_list(f_list:list)-> pd.DataFrame:
    d_list=list(map(clean_excel, f_list))
    return pd.concat(d_list).astype(str)

In [9]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'
bq_client = bigquery.Client()
project_id='pluto-panel'

In [10]:
for k, v in f_dict.items():
    table_id=f'real_estate.{k}_raw'
    print(f'reading year: {k}')
    d=clean_list(v)
    pandas_gbq.to_gbq(d, table_id, project_id=project_id)
    

reading year: 2003


100%|██████████| 1/1 [00:00<00:00, 6754.11it/s]


reading year: 2004


100%|██████████| 1/1 [00:00<00:00, 11275.01it/s]


reading year: 2005


...

100%|██████████| 1/1 [00:00<00:00, 14463.12it/s]


reading year: 2019


100%|██████████| 1/1 [00:00<00:00, 11214.72it/s]


reading year: 2020


100%|██████████| 1/1 [00:00<00:00, 7410.43it/s]
